In [ ]:
import os
import json
from tqdm import tqdm

# 클래스 매핑 딕셔너리
class_mapping = {
    "leye_o": 0,
    "leye_c": 1,
    "reye_o": 2,
    "reye_c": 3
}

# JSON 파일 경로
folder_path = "./valid/crop_json"

# 폴더 내의 파일 순회
file_list = os.listdir(folder_path)
total_files = len(file_list)

# tqdm을 사용하여 진행 상황 시각화
progress_bar = tqdm(total=total_files, desc="진행 중")

# 처리한 파일 개수
processed_files = 0

for file_name in file_list:
    if file_name.endswith(".json"):
        # JSON 파일 경로
        json_file = os.path.join(folder_path, file_name)
        
        # JSON 파일 로드 및 처리
        with open(json_file, "r", encoding="utf-8") as f:
            json_data = json.load(f)
            
        # 이미지 파일 정보
        image_filename = json_data["FileInfo"]["FileName"]
        image_width = int(json_data["FileInfo"]["Width"])
        image_height = int(json_data["FileInfo"]["Height"])

        # 텍스트 파일 경로
        output_folder = "./valid/labels"
        output_filename = os.path.join(output_folder, image_filename.replace(".jpg", ".txt"))

        # 텍스트 파일 생성
        os.makedirs(output_folder, exist_ok=True)  # 출력 폴더가 없을 경우 생성
        with open(output_filename, "w") as output_file:
            # Bounding Box 정보 추출 및 YOLOv5 형식으로 변환
            bbox_info = json_data["ObjectInfo"]["BoundingBox"]

            for obj in ["Leye", "Reye"]:
                obj_info = bbox_info[obj]
                if obj_info["isVisible"]:
                    bbox = [float(coord) for coord in obj_info["Position"]]
                    opened = obj_info.get("Opened", False)  # "Opened" 키가 없을 경우 기본값으로 False 지정
                    class_label = f"{obj.lower()}_{('o' if opened else 'c')}"
                    class_id = class_mapping.get(class_label)  # 클래스 매핑 딕셔너리에서 클래스 ID 가져오기
                    if class_id is not None:  # 클래스 ID가 유효한 경우에만 처리
                        x_center = (bbox[0] + bbox[2]) / (2 * image_width)
                        y_center = (bbox[1] + bbox[3]) / (2 * image_height)
                        width = (bbox[2] - bbox[0]) / image_width
                        height = (bbox[3] - bbox[1]) / image_height

                        # 텍스트 파일에 라벨링 정보 작성
                        line = f"{class_id} {x_center} {y_center} {width} {height}\n"
                        output_file.write(line)
    
    processed_files += 1
    progress_bar.update(1)
    progress_bar.set_postfix({"진행": f"{processed_files}/{total_files}"})

# tqdm 객체 닫기
progress_bar.close()

print("라벨링 파일 생성 완료")